# Import's

In [1]:
import os
import re

from datasets import Dataset, Value
from keras.utils.np_utils import to_categorical

### Variveis Constantes

In [2]:
DIR = "../../"

dataset_path = "data/"
dataset_e_commerce_path = "datasets/dataset_e-commerce/"

### Lendo os Dataset's

In [3]:
dict_datasets : dict[str, Dataset] = dict({})
for arquivo_nome in list(filter(lambda x: x.endswith(".csv"), os.listdir(DIR+dataset_path+dataset_e_commerce_path))):
    dict_datasets.update(dict({arquivo_nome: Dataset.from_csv(DIR+dataset_path+dataset_e_commerce_path+arquivo_nome)}))

Using custom data configuration default-5a1e778d329ff297
Found cached dataset csv (/home/thiago/.cache/huggingface/datasets/csv/default-5a1e778d329ff297/0.0.0)
Using custom data configuration default-cd2eda35244ba4e4
Found cached dataset csv (/home/thiago/.cache/huggingface/datasets/csv/default-cd2eda35244ba4e4/0.0.0)
Using custom data configuration default-68afaead1c193b1b
Found cached dataset csv (/home/thiago/.cache/huggingface/datasets/csv/default-68afaead1c193b1b/0.0.0)
Using custom data configuration default-7769ac3f58b45370
Found cached dataset csv (/home/thiago/.cache/huggingface/datasets/csv/default-7769ac3f58b45370/0.0.0)
Using custom data configuration default-e400a9b2c7782f6f
Found cached dataset csv (/home/thiago/.cache/huggingface/datasets/csv/default-e400a9b2c7782f6f/0.0.0)


In [4]:
dict_datasets

{'utlc_apps.csv': Dataset({
     features: ['original_index', 'review_text', 'review_text_processed', 'review_text_tokenized', 'polarity', 'rating', 'kfold_polarity', 'kfold_rating'],
     num_rows: 1039535
 }),
 'buscape.csv': Dataset({
     features: ['original_index', 'review_text', 'review_text_processed', 'review_text_tokenized', 'polarity', 'rating', 'kfold_polarity', 'kfold_rating'],
     num_rows: 84991
 }),
 'utlc_movies.csv': Dataset({
     features: ['original_index', 'review_text', 'review_text_processed', 'review_text_tokenized', 'polarity', 'rating', 'kfold_polarity', 'kfold_rating'],
     num_rows: 1487449
 }),
 'b2w.csv': Dataset({
     features: ['original_index', 'review_text', 'review_text_processed', 'review_text_tokenized', 'polarity', 'rating', 'kfold_polarity', 'kfold_rating'],
     num_rows: 132373
 }),
 'olist.csv': Dataset({
     features: ['original_index', 'review_text', 'review_text_processed', 'review_text_tokenized', 'polarity', 'rating', 'kfold_polarity'

### Funções

#### Renomeando as colunas

In [5]:
def renomea_colunas(dataset: Dataset):
    dataset = dataset.rename_column("review_text", "text")
    dataset = dataset.rename_column("polarity", "labels")
    dataset = dataset.rename_column("kfold_polarity", "group")
    return dataset

#### Remove as colunas

In [6]:
def remove_colunas(dataset: Dataset):
    dataset = dataset.remove_columns(["original_index", "review_text_tokenized", "review_text_processed", "rating", "kfold_rating"])
    return dataset

#### Convert label

In [7]:
def convert_coluna_to_int(dataset: Dataset, coluna: str):
    novo_tipo = dataset.features.copy()
    novo_tipo["labels"] = Value("int32")
    dataset = dataset.cast(novo_tipo)
    return dataset

#### Convert to categorical

In [8]:
def convert_coluna_to_categorical(data, coluna: str, num_classes: int):
    data[coluna] = to_categorical(data[coluna], num_classes=num_classes)
    return data

#### Convert rating

In [9]:
## Usado para caso for utilizar a coluna 'rating'
def convert_labels(data, coluna: str):
    """
    0 - negativo
    1 - neutro
    2 - positivo
    """
    if data[coluna] <= 2:
        data[coluna] = 0
    elif data[coluna] == 3:
        data[coluna] = 1
    else:
        data[coluna] = 2
    return data

### Filtra Coluna

In [10]:
def filtra_coluna(dataset: Dataset, coluna: str, remover: list = [None, ""]):
    return dataset.filter(lambda dado: dado[coluna] not in remover)

### Limpa texto

In [11]:
def limpa_texto(data, coluna: str):
    if isinstance(data[coluna], str):
        data[coluna] = data[coluna].strip()
    return data

### Arrumando os datasets do e-commerce

#### Automático

In [12]:
for arquivo in dict_datasets.keys():
    print(arquivo)
    dict_datasets[arquivo] = renomea_colunas(dict_datasets[arquivo])
    dict_datasets[arquivo] = remove_colunas(dict_datasets[arquivo])

    dict_datasets[arquivo] = dict_datasets[arquivo].map(lambda dado: limpa_texto(dado, "text"))

    dict_datasets[arquivo] = filtra_coluna(dict_datasets[arquivo], "labels")
    dict_datasets[arquivo] = filtra_coluna(dict_datasets[arquivo], "text")
    
    dict_datasets[arquivo] = convert_coluna_to_int(dict_datasets[arquivo], "labels")
    # dict_datasets[arquivo] = dict_datasets[arquivo].map(lambda data: convert_labels(data, "labels"))
    dict_datasets[arquivo] = dict_datasets[arquivo].add_column("labels_int", dict_datasets[arquivo]["labels"])
    num_classes = len(set(dict_datasets[arquivo]["labels"]))
    dict_datasets[arquivo] = dict_datasets[arquivo].map(lambda batch: convert_coluna_to_categorical(batch, "labels", num_classes), batched=True)
    
    dict_datasets[arquivo].to_json(DIR+dataset_path+"dataset_"+arquivo[:-4]+".json")

Loading cached processed dataset at /home/thiago/.cache/huggingface/datasets/csv/default-5a1e778d329ff297/0.0.0/cache-107169014090089f.arrow
Loading cached processed dataset at /home/thiago/.cache/huggingface/datasets/csv/default-5a1e778d329ff297/0.0.0/cache-4e777ccb835b9077.arrow
Loading cached processed dataset at /home/thiago/.cache/huggingface/datasets/csv/default-5a1e778d329ff297/0.0.0/cache-a4c69ee8e3c3d403.arrow
Loading cached processed dataset at /home/thiago/.cache/huggingface/datasets/csv/default-5a1e778d329ff297/0.0.0/cache-a42a53564de2ea0d.arrow


utlc_apps.csv


  0%|          | 0/969 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/97 [00:00<?, ?ba/s]

buscape.csv


...

  0%|          | 0/42 [00:00<?, ?ba/s]

  0%|          | 0/39 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/39 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

#### Manual

In [ ]:
dict_datasets.keys()

In [ ]:
dict_datasets.keys()

In [ ]:
dict_datasets.keys()

In [ ]:
dict_datasets.keys()

In [ ]:
dataset_str = "olist.csv"

In [ ]:
dict_datasets[dataset_str]

##### Renomea colunas

In [ ]:
dict_datasets[dataset_str].features

In [ ]:
dict_datasets[dataset_str] = renomea_colunas(dict_datasets[dataset_str])

In [ ]:
dict_datasets[dataset_str].features

##### Remove colunas

In [ ]:
dict_datasets[dataset_str].features

In [ ]:
dict_datasets[dataset_str] = remove_colunas(dict_datasets[dataset_str])

In [ ]:
dict_datasets[dataset_str].features

##### Remove valores None

Coluna labels

In [ ]:
set(dict_datasets[dataset_str]["labels"])

In [ ]:
None in dict_datasets[dataset_str]["labels"]

In [ ]:
dict_datasets[dataset_str] = remove_None(dict_datasets[dataset_str], "labels")

In [ ]:
None in dict_datasets[dataset_str]["labels"]

In [ ]:
set(dict_datasets[dataset_str]["labels"])

Coluna text

In [ ]:
None in dict_datasets[dataset_str]["text"]

In [ ]:
dict_datasets[dataset_str] = remove_None(dict_datasets[dataset_str], "text")

In [ ]:
None in dict_datasets[dataset_str]["text"]

Coluna Group

In [ ]:
set(dict_datasets[dataset_str]["group"])

In [ ]:
None in dict_datasets[dataset_str]["group"]

In [ ]:
dict_datasets[dataset_str] = remove_None(dict_datasets[dataset_str], "group")

In [ ]:
None in dict_datasets[dataset_str]["group"]

In [ ]:
set(dict_datasets[dataset_str]["group"])

##### Convert para int

In [ ]:
dict_datasets[dataset_str].features

In [ ]:
dict_datasets[dataset_str] =  convert_coluna_to_int(dict_datasets[dataset_str], "labels")

In [ ]:
dict_datasets[dataset_str].features

##### Convert para to_categorical

In [ ]:
dict_datasets[dataset_str].features

In [ ]:
dict_datasets[dataset_str]["labels"][:5]

In [ ]:
set(dict_datasets[dataset_str]["labels"])

In [ ]:
dict_datasets[dataset_str] = dict_datasets[dataset_str].map(lambda batch: convert_coluna_to_categorical(batch, "labels", len(set(dict_datasets[dataset_str]["labels"]))), batched=True)

In [ ]:
dict_datasets[dataset_str].features

In [ ]:
dict_datasets[dataset_str]["labels"][:5]

##### Verificando os grupos

In [ ]:
def group_by(dataset: Dataset, coluna: str):
    lista_grupos = []
    for grupo in set(dataset[coluna]):
        lista_grupos.append(dataset.filter(lambda x: x[coluna]==grupo))
    return lista_grupos

In [ ]:
lista_datasets = group_by(dict_datasets[dataset_str], "group")

In [ ]:
for index, dataset_part in enumerate(lista_datasets):
    print(str(index) + " - " + str(len(dataset_part)))

##### Salvando o dataset

In [ ]:
dict_datasets[dataset_str].to_json(DIR+dataset_path+"dataset_"+dataset_str+".json")